<a href="https://colab.research.google.com/github/SJSUMS/PCam/blob/main/255PCam_mobileNetV2_mavis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow import keras
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing import image
from sklearn import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, datetime, time

path = '/content/drive/Shareddrives/data 255 Deep Learning/'

## sample data

testing set (30k) as our sample dataset for train/test/val sets.

In [ ]:
# load Pre-processed X and y (32768)
from pickle import load
X = load(open(path + 'data/X.pkl', 'rb'))
y = load(open(path + 'data/y.pkl', 'rb'))

In [ ]:
print(y.shape)

(32768, 2)


### split dataset

* Training/Testing = 80/10
* Validation = 5000 out of training

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# reserve 5000 samples for validation
X_val = X_train[-5000:]
y_val = y_train[-5000:]
X_train = X_train[:-5000]
y_train = y_train[:-5000]

In [ ]:
print(X_train.shape)
print(y_train.shape)

print(X_val.shape)
print(y_val.shape)

print(X_test.shape)
print(y_test.shape)


(24491, 96, 96, 3)
(24491, 2)
(5000, 96, 96, 3)
(5000, 2)
(3277, 96, 96, 3)
(3277, 2)


# MobileNet2

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import callbacks, layers, Input
from keras.models import Model
from tensorflow.keras.models import Sequential

### MobileNetV2

In [ ]:
# pretrained mobeilNetV2 with imagenet weights
def feature_extractor(inputs):
  feature_extractor_layer = MobileNetV2(
      input_shape = (96,96,3),
      include_top = False,
      weights='imagenet')(inputs)
  return feature_extractor_layer

In [ ]:
# classifier
def classifier(inputs):
  x = layers.GlobalAveragePooling2D()(inputs)
  x = layers.Flatten()(x)
  
  # add 2 dense layers
  x = layers.Dense(512, activation='relu')(x)
  x = layers.Dense(128, activation='relu')(x)
  
  output = layers.Dense(2, activation='sigmoid')(x)
  return output

In [ ]:
# build model
def mobileNetVar(inputs):
    
  feature_mobileNet = feature_extractor(inputs)
  class_output = classifier(feature_mobileNet)

  model = Model(inputs=inputs, outputs=class_output, name='base_mobileNet')

  # compile
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  
  return model

In [ ]:
# define input layer
inputs = Input(shape=(96,96,3))

model = mobileNetVar(inputs)
model.summary()

Model: "base_mobileNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 mobilenetv2_1.00_96 (Functi  (None, 3, 3, 1280)       2257984   
 onal)                                                           
                                                                 
 global_average_pooling2d_1   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten_1 (Flatten)         (None, 1280)              0         
                                                                 
 dense_3 (Dense)             (None, 512)               655872    
                                                                 
 dense_4 (Dense)             (None, 128)            

#### Training

In [ ]:
# checkpoints
checkpoint_filepath = path + 'checkpoints'

model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath, 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=True, 
    save_freq='epoch')

# train
ep = 20
batch = 128

start = time.time()
# use validation set to validate
hist = model.fit(X_train, y_train, batch_size=batch, epochs=ep, 
                 verbose = 1, validation_data=(X_val,y_val), shuffle=True, 
                 callbacks=[model_checkpoint_callback])
stop = time.time()
print(f"Training time: {stop - start}s")

Epoch 1/20
192/192 [==============================] - ETA: 0s - loss: 0.2483 - accuracy: 0.9020

In [ ]:
# evaluate model loss & accuracy on test set
score = model.evaluate(X_test,y_test, batch_size=128)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

NameError: ignored

In [ ]:
from keras.models import load_model, save_model

# save model
model.save(path + "models")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/PCam/models/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)
